In [ ]:
!mkdir -p /root/.config/kaggle
! mv kaggle.json /root/.config/kaggle/kaggle.json
!chmod 600 /root/.config/kaggle/kaggle.json
!kaggle competitions download -c udea-ai-4-eng-20251-pruebas-saber-pro-colombia
!unzip udea-ai-4-eng-20251-pruebas-saber-pro-colombia.zip

  0% 0.00/29.9M [00:00<?, ?B/s]
100% 29.9M/29.9M [00:00<00:00, 1.09GB/s]
Archive:  udea-ai-4-eng-20251-pruebas-saber-pro-colombia.zip
  inflating: submission_example.csv  
  inflating: test.csv                
  inflating: train.csv               


In [1]:
import pandas as pd
import numpy as np

## Exploración de datos y valores NAN

### cantidad de valores NAN por columna

In [ ]:
pd.read_csv('train.csv').isna().sum()

,0
ID,0
PERIODO,0
ESTU_PRGM_ACADEMICO,0
ESTU_PRGM_DEPARTAMENTO,0
ESTU_VALORMATRICULAUNIVERSIDAD,6287
ESTU_HORASSEMANATRABAJA,30857
FAMI_ESTRATOVIVIENDA,32137
FAMI_TIENEINTERNET,26629
FAMI_EDUCACIONPADRE,23178
FAMI_TIENELAVADORA,39773


### Valores únicos por columna

In [ ]:
df = pd.read_csv('train.csv')
df.nunique()

,0
ID,692500
PERIODO,9
ESTU_PRGM_ACADEMICO,948
ESTU_PRGM_DEPARTAMENTO,31
ESTU_VALORMATRICULAUNIVERSIDAD,8
ESTU_HORASSEMANATRABAJA,5
FAMI_ESTRATOVIVIENDA,7
FAMI_TIENEINTERNET,2
FAMI_EDUCACIONPADRE,12
FAMI_TIENELAVADORA,2


### Descripcción de los datos

In [ ]:
df._get_numeric_data().describe().T

,count,mean,std,min,25%,50%,75%,max
ID,692500.0,494606.130576,285585.209455,1.0,247324.750,494564.500,741782.500,989286.000
PERIODO,692500.0,20198.366679,10.535037,20183.0,20195.000,20195.000,20203.000,20213.000
coef_1,692500.0,0.268629,0.122130,0.0,0.203,0.240,0.314,0.657
coef_2,692500.0,0.259996,0.093480,0.0,0.212,0.271,0.309,0.487
coef_3,692500.0,0.262087,0.058862,0.0,0.254,0.276,0.293,0.320
coef_4,692500.0,0.262903,0.067944,0.0,0.255,0.285,0.303,0.332


## Lectura de datasets y asignación a variables

In [2]:
dtr = pd.read_csv("train.csv")
dts = pd.read_csv("test.csv")
lentr = len(dtr)
dtr.shape, dts.shape

((692500, 21), (296786, 20))

## Limpieza y reemplazo de valores NAN para la columna ESTU_HORASSEMANATRABAJA

In [3]:
def organizar_horas_trabajadas_estudiante(data, columna):

    # Valores posibles
    mapeo = {
        "0": 0,
        "menos de 10 horas": 1,
        "entre 11 y 20 horas": 2,
        "entre 21 y 30 horas": 3,
        "más de 30 horas": 4
    }

    # Rellenar NaNs con valores  válidos
    modas = data.groupby('FAMI_ESTRATOVIVIENDA')[columna].transform(lambda x: x.mode().iloc[0])
    data[columna] = data[columna].fillna(modas)

    # Pasar todo a minúsculas y quitar espacios extra
    data[columna] = data[columna].astype(str).str.strip().str.lower()

    # Aplicar mapeo numérico
    data[columna] = data[columna].map(mapeo)

    return data


## Limpieza y reemplzado de valores NAN para la columna ESTU_HORASSEMANATRABAJA utilizando dummies

In [4]:
def organizar_horas_trabajadas_estudiante_columna(data, columna):

    # Valores posibles ordenados
    mapeo = {
        "0": 0,
        "menos de 10 horas": 1,
        "entre 11 y 20 horas": 2,
        "entre 21 y 30 horas": 3,
        "más de 30 horas": 4
    }

    # Rellenar NaNs con valores cíclicos válidos
    modas = data.groupby('FAMI_ESTRATOVIVIENDA')[columna].transform(lambda x: x.mode().iloc[0])
    data[columna] = data[columna].fillna(modas)

    dummies = pd.get_dummies(data[columna], prefix="Trabaja").astype(int)
    data = data.drop(columns=[columna]).join(dummies)

    return data

## Limpieza y reemplazo de valores NAN para la columna ESTU_PRGM_ACADEMICO

In [5]:
def organizar_programa_estudiante(data, columna):
    # Diccionario con categorías y palabras clave asociadas
    categorias = {
        "ingenieria": ["ingenier"],
        "salud": ["medicina", "enfermer", "fisioterap", "odontolog", "nutric", "terapia", "bacteriolog", "microbiolog", "farmac", "optometr", "salud"],
        "sociales": ["psicolog", "trabajo social", "sociolog","histor" "crimin", "politic", "juris", "derecho", "filosof","gobier","histori"],
        "administracion": ["admin", "gestión", "gestion", "logist", "negocios","direcc","gerenc"],
        "economia_finanzas": ["finanzas", "econom", "comercio", "merca", "marketing"],
        "comunicacion": ["comunic", "periodismo", "medios", "publicidad","cine", "telev","fotogra","audio","publi"],
        "arte": ["arte", "musica", "teatro", "danza", "literatura", "bellas artes"],
        "educacion": ["licenc", "pedagog", "educac", "docencia","profes"],
        "diseño": ["dise","diseño"],
        "matematica": ["matema", "matemá", "estadis", "estadísti","arquite"]
    }

    # Normaliza los textos
    data[columna] = data[columna].astype(str).str.lower().str.strip()

    # Función interna para aplicar a cada valor
    def clasificar(programa):
        for categoria, palabras_clave in categorias.items():
            if any(palabra in programa for palabra in palabras_clave):
                return categoria
        return "otras"

    # Aplicar categorización
    data[columna] = data[columna].apply(clasificar)

    dummies = pd.get_dummies(data[columna], prefix="Programa").astype(int)
    data = data.drop(columns=[columna]).join(dummies)

    return data  # ✅ ESTA LÍNEA ES OBLIGATORIA



## Limpieza y reemplzado de valores NAN para la columna ESTU_PRGM_DEPARTAMENTO

In [6]:
def organizar_departamentos(data, columna):
  dummies = pd.get_dummies(data[columna], prefix="DEPARTAMENTO").astype(int)
  data = data.drop(columns=[columna]).join(dummies)

  return data

##  Limpieza y reemplazo de valores NAN para la columna FAMI_ESTRATOVIVIENDA

In [7]:
def organizar_estratos(data, columna):
  data[columna] = (
      data[columna]
      .str.lower()
      .str.replace("sin estrato", "0", regex=False)
      .str.replace("estrato ", "", regex=False))

  indices_nan = data[data[columna].isna()].index

  moda = data[columna].mode().values[0]

  data[columna] = data[columna].fillna(moda)


  data[columna] = (
    data[columna]
    .astype(int))

  return data

## Limpieza y reemplzado de valores NAN para las columnas que tienen solo dos posibles valores

In [8]:
def resumen_columna_binaria(data, columna, valores):
    si = (data[columna] == valores[0]).sum()
    no = (data[columna] == valores[1]).sum()
    nan = data[columna].isna().sum()
    total = len(data)

    si_pct = (si * 100) / total
    no_pct = (no * 100) / total
    nan_pct = (nan * 100) / total

    return (si_pct / 100), (nan_pct / 100)


def organizar_columna_binario_con_proporcion(data, columna, valores):
    n_missing = data[columna].isna().sum()

    if n_missing == 0:
        data[columna] = (data[columna]
                         .str.replace(valores[0], "1", regex=False)
                         .str.replace(valores[1], "0", regex=False)
                         .astype(int))
        print(f"No hay valores faltantes en la columna '{columna}'.")
        return data

    si_pct, nan_pct = resumen_columna_binaria(data, columna, valores)

    # Distribuir los NaNs de forma proporcional entre "sí" y "no"
    si = si_pct + (nan_pct / 2)
    no = 1 - si
    proporciones = [si, no]

    # Rellenar con valores aleatorios
    valores_aleatorios = np.random.choice(valores, size=n_missing, p=proporciones)
    data.loc[data[columna].isna(), columna] = valores_aleatorios

    # Convertir texto a binario
    data[columna] = (data[columna]
                     .str.replace(valores[0], "1", regex=False)
                     .str.replace(valores[1], "0", regex=False)
                     .astype(int))

    return data



## Limpieza y reemplazo de valores NAN para la columna ESTU_VALORMATRICULAUNIVERSIDAD

In [9]:
def organizar_valor_matricula_en_una_columna(data, columna):
    mapeo = {
       'Más de 7 millones':6,
       'Entre 5.5 millones y menos de 7 millones':5,
       'Entre 4 millones y menos de 5.5 millones':4,
       'Entre 2.5 millones y menos de 4 millones':3,
       'Entre 1 millón y menos de 2.5 millones':2,
       'Entre 500 mil y menos de 1 millón':1,
       'Menos de 500 mil': 0,
       'No pagó matrícula':0
      }

    # Rellenar NaNs con valores cíclicos válidos
    modas = data.groupby('FAMI_ESTRATOVIVIENDA')[columna].transform(lambda x: x.mode().iloc[0])
    data[columna] = data[columna].fillna(modas)

    # Aplicar mapeo numérico
    data[columna] = data[columna].map(mapeo)

    return data


## Limpieza y reemplazo de valores NAN para la columna FAMI_EDUCACIONPADRE

In [10]:
def organizar_educacion_padres(data, columna):
  # Lista de estratos (como strings, si así están en el DataFrame)

  mapeo = {
    'No sabe':0,
    'Ninguno':0,
    'No Aplica':0,
    'Primaria incompleta':1,
    'Primaria completa':1,
    'Secundaria (Bachillerato) incompleta':2,
    'Secundaria (Bachillerato) completa':3,
    'Técnica o tecnológica incompleta':4,
    'Técnica o tecnológica completa':5,
    'Educación profesional incompleta':5,
    'Educación profesional completa':6,
    'Postgrado':7,
  }

  # Asignar uno por uno, reiniciando al llegar al final de la lista de estratos
  modas = data.groupby('FAMI_ESTRATOVIVIENDA')[columna].transform(lambda x: x.mode().iloc[0])
  data[columna] = data[columna].fillna(modas)

  data[columna] = data[columna].map(mapeo)

  return data


## Limpieza y reemplazo de valores NAN para la columna PERIODO

In [11]:
def organizar_periodos(data, columna):
    mapeo = {
        20183:0,
        20184:0, 
        20194:1,
        20195:1,
        20196:1,
        20202:2,
        20203:2,
        20212:3,
        20213:3
    }

    data[columna] = data[columna].map(mapeo)

    return data

## Limpieza y reemplazo de valores NAN para la columna FAMI_EDUCACIONMADRE

## Análisis de la correclación entre las columnas de coeficientes

In [ ]:
coef_cols = ['coef_1', 'coef_2', 'coef_3', 'coef_4']
dtr_coef = dtr[coef_cols]

correlation_matrix = dtr_coef.corr()

print("Matriz de correlación entre las columnas 'coef':")
print(correlation_matrix)

Matriz de correlación entre las columnas 'coef':
          coef_1    coef_2    coef_3    coef_4
coef_1  1.000000 -0.420428  0.401190 -0.179439
coef_2 -0.420428  1.000000  0.356097  0.821443
coef_3  0.401190  0.356097  1.000000  0.726692
coef_4 -0.179439  0.821443  0.726692  1.000000


## Función donde se eliminan columnas innecesarias y se aplican las funciones de limpieza

In [12]:
def clean_data(data):
  columnas_inecesarias = [
      "ID",
      "FAMI_TIENELAVADORA",
      "coef_4",
      # "coef_1",
      #"PERIODO",
      # "FAMI_TIENEAUTOMOVIL",
      "FAMI_TIENEINTERNET.1",
      "ESTU_PRIVADO_LIBERTAD",
      #"ESTU_HORASSEMANATRABAJA",
      # "FAMI_EDUCACIONPADRE",
      # "FAMI_EDUCACIONMADRE",
      #"ESTU_PRGM_ACADEMICO",
      #"ESTU_PRGM_DEPARTAMENTO",
      #"FAMI_TIENEINTERNET",
      #"FAMI_TIENECOMPUTADOR",
      #"ESTU_PAGOMATRICULAPROPIO",
      #"ESTU_VALORMATRICULAUNIVERSIDAD"
  ]
  data=data.drop(columns=columnas_inecesarias)

  data=organizar_estratos(data, "FAMI_ESTRATOVIVIENDA")
  data=organizar_periodos(data, "PERIODO")
  data=organizar_horas_trabajadas_estudiante(data, "ESTU_HORASSEMANATRABAJA")
  data=organizar_programa_estudiante(data, "ESTU_PRGM_ACADEMICO")
  data=organizar_departamentos(data, "ESTU_PRGM_DEPARTAMENTO")
  data=organizar_columna_binario_con_proporcion(data, "FAMI_TIENEINTERNET", ["Si", "No"])
  data=organizar_columna_binario_con_proporcion(data, "FAMI_TIENECOMPUTADOR", ["Si", "No"])
  # data=organizar_columna_binario_con_proporcion(data, "FAMI_TIENELAVADORA", ["Si", "No"])
  data=organizar_columna_binario_con_proporcion(data, "FAMI_TIENEAUTOMOVIL", ["Si", "No"])
  data=organizar_columna_binario_con_proporcion(data, "ESTU_PAGOMATRICULAPROPIO", ["Si", "No"])

  data=organizar_valor_matricula_en_una_columna(data, "ESTU_VALORMATRICULAUNIVERSIDAD")
  data=organizar_educacion_padres(data, "FAMI_EDUCACIONPADRE")
  data=organizar_educacion_padres(data, "FAMI_EDUCACIONMADRE")
  return data

In [13]:
source_cols = [i for i in dtr.columns if i!="RENDIMIENTO_GLOBAL"]
all_data = pd.concat((dtr[source_cols], dts[source_cols]))
all_data.index = range(len(all_data))
all_data = clean_data(all_data)

Xtr, ytr = all_data.iloc[:lentr].values, dtr["RENDIMIENTO_GLOBAL"].values
Xts      = all_data.iloc[lentr:].values
print (Xtr.shape, ytr.shape)
print (Xts.shape)

(692500, 55) (692500,)
(296786, 55)


In [ ]:
all_data.head()

,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_EDUCACIONMADRE,coef_1,coef_4,...,DEPARTAMENTO_NORTE SANTANDER,DEPARTAMENTO_PUTUMAYO,DEPARTAMENTO_QUINDIO,DEPARTAMENTO_RISARALDA,DEPARTAMENTO_SAN ANDRES,DEPARTAMENTO_SANTANDER,DEPARTAMENTO_SUCRE,DEPARTAMENTO_TOLIMA,DEPARTAMENTO_VALLE,DEPARTAMENTO_VAUPES
0,5,1,3,1,0,0,1,8,0.322,0.267,...,0,0,0,0,0,0,0,0,0,0
1,3,0,3,0,1,0,1,0,0.311,0.264,...,0,0,0,0,0,0,0,0,0,0
2,3,4,3,1,2,0,0,2,0.297,0.264,...,0,0,0,0,0,0,0,0,0,0
3,4,0,4,1,3,0,1,2,0.485,0.190,...,0,0,0,0,0,1,0,0,0,0
4,3,3,3,1,4,0,1,4,0.316,0.294,...,0,0,0,0,0,0,0,0,0,0


## Random Forest Classifier

In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

Xtrain, Xtest, ytrain, ytest = train_test_split(Xtr,ytr, test_size=0.2)

param_grid = {
    "n_estimators": [160, 170, 180],
    "max_depth": [16, 18, 20],
}

grid_search = GridSearchCV(estimator=RandomForestClassifier( n_jobs=-2), param_grid=param_grid, cv=3)
grid_search.fit(Xtrain, ytrain)

,estimator,RandomForestC...ier(n_jobs=-2)
,param_grid,"{'max_depth': [16, 18, ...], 'n_estimators': [160, 170, ...]}"
,scoring,None
,n_jobs,None
,refit,True
,cv,3
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,n_estimators,170


In [15]:
grid_search.best_params_

{'max_depth': 18, 'n_estimators': 170}

In [16]:
grid_search.best_estimator_.score(Xtrain, ytrain), grid_search.best_estimator_.score(Xtest, ytest)

(0.6155685920577617, 0.41101083032490976)

In [17]:
pd.DataFrame({'ID': dts['ID'].values, 'RENDIMIENTO_GLOBAL': grid_search.best_estimator_.predict(Xts)}).to_csv('predict.csv', index=False)